In [ ]:
!pip3 install fastFM

In [34]:
import os
import numpy as np
import pandas as pd
import fastFM
from fastFM.datasets import make_user_item_regression
from fastFM import sgd
from fastFM import als
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
from fastFM import mcmc
import functools as fct
import itertools as itools
import random, scipy

In [35]:
train_df = pd.read_csv("training_set.csv")
train_df.head()

,userID,itemID,styleID,rating
0,401,32780,12224,3.0
1,7548,21688,9020,3.5
2,7663,1521,568,4.0
3,1357,3824,1417,3.5
4,1361,727,263,4.5


In [36]:
items_styles = pd.read_csv("items_styles.csv")
items_styles.head()

,itemID,styleID
0,11757,1199
1,5441,1199
2,19960,1199
3,55900,1199
4,57110,14879


## Convertir a formato fastFM

In [37]:
def get_single_entries_in_fm_input_format(data, itemlist):

    '''Cree el formato de entrada necesario (datos, (fila, columna)) para la matriz csc para
    las entradas individuales en los datos. Cada entrada ocuparía una fila. Esto significa que
    daría como resultado una matriz csc con dimensión (| datos | x | lista de elementos |).
    '''

    column = len(itemlist)
    row = len(data)
    shape = (row, column)

    row_inds = np.zeros(len(data), dtype=np.int32)
    col_inds = np.zeros(len(data), dtype=np.int32)
    datalist = np.zeros(len(data), dtype=np.float32)

    for i in range(len(data)):
        item = data[i]
        val = 1
        datalist[i] = val

        # ubica su posición en la lista de elementos, arroja un error si el elemento no es un
        # artículo posible
        col_ind = np.where(itemlist==item)[0]

        # no deben ser elementos duplicados en la lista de elementos
        assert len(col_ind) == 1
        col_ind = col_ind[0]
        row_ind = i

        col_inds[i] = col_ind
        row_inds[i] = row_ind

    return datalist, row_inds, col_inds, shape


def get_multi_entries_in_fm_input_format(data, itemlist, norm_func=None):

    '''Cree el formato de entrada necesario (datos, (fila, columna)) para la matriz csc para
    las entradas múltiples en los datos. Cada conjunto de entradas múltiples ocuparía una fila.
    Esto significa que daría como resultado una matriz csc con dimensión
    (| conjuntos de entradas en datos | x | lista de elementos |).
    '''

    column = len(itemlist)

    # número de conjuntos de entradas en los datos
    row = len(data)
    shape = (row, column)

    # numero de datos
    num_of_data = fct.reduce(lambda x, y: x + len(y), data, 0)
    row_inds = np.zeros(num_of_data, dtype=np.int32)
    col_inds = np.zeros(num_of_data, dtype=np.int32)
    datalist = np.zeros(num_of_data, dtype=np.float32)
    cnt = 0
    for i in range(len(data)):
        multi_entry = data[i]

        if norm_func != None:
            # función que recibe el tamaño del multi_entry para decidir cómo normalizarlo
            val = norm_func(len(multi_entry))
        else:
            # asignación de valor binario por defecto
            val = 1 if len(multi_entry) > 0 else 0

        # para cada entrada en multi_entry, ubique su posición en la lista de elementos,
        # arroja error si el elemento no es un elemento posible
        # todas las entradas permanecen en la misma fila
        row_ind = i
        for item in multi_entry:
            col_ind = np.where(itemlist==item)[0]
            assert len(col_ind) == 1
            col_ind = col_ind[0]

            datalist[cnt] = val
            col_inds[cnt] = col_ind
            row_inds[cnt] = row_ind

            # actualiza contador
            cnt += 1

    return datalist, row_inds, col_inds, shape

### Conversión de los datos

In [38]:
beerlist = items_styles.sort_values('itemID')['itemID'].unique()
userlist = train_df.sort_values('userID')['userID'].unique()
stylelist = items_styles.sort_values('styleID')['styleID'].unique()


# usuarios que dieron ratings
user_data = train_df['userID'].values

# items que recibieron ratings
beer_data = train_df['itemID'].values

# data de estilo de cerveza
styles_data = train_df['styleID'].values

# target vector: ratings
rating_data = train_df['rating'].values


# convertir a formato fastFM utilizando funciones de arriba
user_datalist, user_row_inds, user_col_inds, user_shape = get_single_entries_in_fm_input_format(data=user_data,
                                                                                                itemlist=userlist)

beer_datalist, beer_row_inds, beer_col_inds, beer_shape = get_single_entries_in_fm_input_format(data=beer_data,
                                                                                                   itemlist=beerlist)

style_datalist, style_row_inds, style_col_inds, style_shape = get_single_entries_in_fm_input_format(data=styles_data,
                                                                                                   itemlist=stylelist)

# Concatena las dos columnas cambiando los índices de las columnas relacionadas con beer.
# cambiar por el número de columnas en las columnas de usuario
shift_by = len(userlist)
beer_col_inds += shift_by

# concatena los datos (agregamos item_styles)
datalist = np.append(user_datalist, [beer_datalist, style_datalist])
row_inds = np.append(user_row_inds, [beer_row_inds, style_row_inds])
col_inds = np.append(user_col_inds, [beer_col_inds,style_col_inds])

# asegúrese de que ambos conjuntos de características tengan el mismo número de filas
print('User feature set shape: {}\nItem feature set shape: {}\nStyle feature set shape: {}'.format(user_shape, beer_shape, style_shape))

assert user_shape[0] == beer_shape[0]
shape = (user_shape[0], user_shape[1] + beer_shape[1] + style_shape[1])
print('Dimension of FM input: {}'.format(shape))


X_train = csc_matrix((datalist, (row_inds, col_inds)), shape=shape)
y_train = rating_data

User feature set shape: (35534, 8320)
Item feature set shape: (35534, 1836)
Style feature set shape: (35534, 210)
Dimension of FM input: (35534, 10366)


In [39]:
validation_df = pd.read_csv("validation_set.csv")
validation_df.head()

,userID,itemID,styleID,rating
0,1040,4920,1315,4.0
1,1306,7348,1199,4.0
2,932,4073,1199,4.0
3,6735,16074,1199,4.5
4,197,39299,3268,4.0


In [40]:
# usuarios que dieron ratings
user_data = validation_df['userID'].values

# items que recibieron ratings
beer_data = validation_df['itemID'].values

# data de estilo de cerveza
styles_data = validation_df['styleID'].values

# target vector: ratings
rating_data = validation_df['rating'].values


# convertir a formato fastFM utilizando funciones de arriba
user_datalist, user_row_inds, user_col_inds, user_shape = get_single_entries_in_fm_input_format(data=user_data,
                                                                                                itemlist=userlist)

beer_datalist, beer_row_inds, beer_col_inds, beer_shape = get_single_entries_in_fm_input_format(data=beer_data,
                                                                                                   itemlist=beerlist)

style_datalist, style_row_inds, style_col_inds, style_shape = get_single_entries_in_fm_input_format(data=styles_data,
                                                                                                   itemlist=stylelist)

# Concatena las dos columnas cambiando los índices de las columnas relacionadas con beer.
# cambiar por el número de columnas en las columnas de usuario
shift_by = len(userlist)
beer_col_inds += shift_by

# concatena los datos (agregamos item_styles)
datalist = np.append(user_datalist, [beer_datalist, style_datalist])
row_inds = np.append(user_row_inds, [beer_row_inds, style_row_inds])
col_inds = np.append(user_col_inds, [beer_col_inds,style_col_inds])

# asegúrese de que ambos conjuntos de características tengan el mismo número de filas
print('User feature set shape: {}\nItem feature set shape: {}\nStyle feature set shape: {}'.format(user_shape, beer_shape, style_shape))

assert user_shape[0] == beer_shape[0]
shape = (user_shape[0], user_shape[1] + beer_shape[1] + style_shape[1])
print('Dimension of FM input: {}'.format(shape))

X_validation = csc_matrix((datalist, (row_inds, col_inds)), shape=shape)
y_validation = rating_data

User feature set shape: (8845, 8320)
Item feature set shape: (8845, 1836)
Style feature set shape: (8845, 210)
Dimension of FM input: (8845, 10366)


In [41]:
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=10, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)

FMRegression(l2_reg_V=0.5, n_iter=1000, rank=10)

In [42]:
y_pred = fm.predict(X_validation)

In [43]:
from sklearn.metrics import mean_squared_error

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse = root_mean_squared_error(y_validation, y_pred)
print(f"RMSE: {rmse}")

RMSE: 0.8378507457519454


In [49]:
def parse_data_for_user(user_id):
  beer_data = items_styles["itemID"].values
  user_data = [user_id] * len(beer_data)
  styles_data = items_styles["styleID"].values

  # convertir a formato fastFM utilizando funciones de arriba
  user_datalist, user_row_inds, user_col_inds, user_shape = get_single_entries_in_fm_input_format(data=user_data,
                                                                                                  itemlist=userlist)

  beer_datalist, beer_row_inds, beer_col_inds, beer_shape = get_single_entries_in_fm_input_format(data=beer_data,
                                                                                                    itemlist=beerlist)

  style_datalist, style_row_inds, style_col_inds, style_shape = get_single_entries_in_fm_input_format(data=styles_data,
                                                                                                    itemlist=stylelist)

  # Concatena las dos columnas cambiando los índices de las columnas relacionadas con beer.
  # cambiar por el número de columnas en las columnas de usuario
  shift_by = len(userlist)
  beer_col_inds += shift_by

  # concatena los datos (agregamos item_styles)
  datalist = np.append(user_datalist, [beer_datalist, style_datalist])
  row_inds = np.append(user_row_inds, [beer_row_inds, style_row_inds])
  col_inds = np.append(user_col_inds, [beer_col_inds,style_col_inds])

  # asegúrese de que ambos conjuntos de características tengan el mismo número de filas

  assert user_shape[0] == beer_shape[0]
  shape = (user_shape[0], user_shape[1] + beer_shape[1] + style_shape[1])

  X_test = csc_matrix((datalist, (row_inds, col_inds)), shape=shape)
  return X_test

In [54]:
def get_top_n_recommendations(model, user_id, n=10):
    X_test = parse_data_for_user(user_id)
    y_test = model.predict(X_test)
    predictions = [(beer_data[i], y_test[i]) for i in range(len(y_test))]
    predictions.sort(key=lambda x: x[1], reverse=True)
    return [prediction[0] for prediction in predictions[:n]]

In [55]:
ranking = get_top_n_recommendations(fm, 1040)
print(ranking)

[17538, 48911, 21822, 436, 20470, 11757, 429, 60511, 21822, 1504]


## Ranking con datos de testeo

In [ ]:
import json

test = json.load(open(os.path.join("ranking_template_fill.json")))

In [ ]:
for user in test.keys():
    user_id = int(user)
    ranking = get_top_n_recommendations(fm, user_id)
    ranking = [int(item) for item in ranking]
    test[user] = ranking

In [ ]:
with open(os.path.join("fm_ranking.json"), "w") as file:
    json.dump(test, file, indent=4)